# Imports

In [1]:
# !pip install a2pm

  Obtaining dependency information for a2pm from https://files.pythonhosted.org/packages/b3/68/75bee5fae78bd4815653b095a50b5c534fd14dd7a9545dea77bb796d840c/a2pm-1.2.0-py3-none-any.whl.metadata


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
import torch

import time

In [2]:
from a2pm import A2PMethod
from a2pm.callbacks import BaseCallback, MetricCallback, TimeCallback
from a2pm.patterns import BasePattern, CombinationPattern, IntervalPattern
from a2pm.wrappers import BaseWrapper, KerasWrapper, SklearnWrapper, TorchWrapper

In [3]:
# X_train = np.load('X-IIoT-pre-processed/X_train.npy')
# y_train = np.load('X-IIoT-pre-processed/y_train.npy')
# X_val = np.load('X-IIoT-pre-processed/X_val.npy')
# y_val = np.load('X-IIoT-pre-processed/y_val.npy')
# X_test = np.load('X-IIoT-pre-processed/X_test.npy')
# y_test = np.load('X-IIoT-pre-processed/y_test.npy')

X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

# Train

We train the baseline model to get an impression of how our model perform on an IDS dataset without any adversarial samples.

In [4]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping


# from sklearn.model_selection import GridSearchCV

2024-07-17 19:52:24.669691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 19:52:24.688908: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 19:52:24.694803: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 19:52:24.711006: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 19:52:25.583487: W tensorflow/compiler/tf2

In [ ]:
# model_xgb = XGBClassifier()

# param_grid = {
#     # 'n_estimators': [10, 50, 100],
#     # 'max_depth': [None, 50, 100],
#     # 'n_estimators': [10],
#     # 'max_depth': [None],
# }

# grid_search_xgb = GridSearchCV(model_xgb, param_grid, cv=3, scoring='accuracy', verbose=3)
# grid_search_xgb.fit(X_train, y_train)

# print("Best parameters found: ", grid_search_xgb.best_params_)

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))


dnn_model = Sequential()
dnn_model.add(Input(shape=input_shape))
dnn_model.add(Dense(units=30, activation='relu'))
dnn_model.add(Dense(units=20, activation='relu'))
dnn_model.add(Dense(units=num_classes, activation='softmax'))

opt = SGD(learning_rate=0.01)

dnn_model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
hist = dnn_model.fit(X_train, y_train, epochs=20, batch_size=64,
                     validation_data=(X_val, y_val),
                     callbacks=[early_stopping])

# test_loss, test_acc = dnn_model.evaluate(X_test, y_test)
# print('Test accuracy:', test_acc)

# y_hat = dnn_model.predict(X_test)
# y_hat = np.argmax(y_hat, axis=-1)

2024-07-17 19:52:27.506984: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 75952 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:46:00.0, compute capability: 8.0


Epoch 1/20


# Evaluate

Evaluate the RF model on the IDS dataset, once again with no adversarial samples.

## Helpers

In [5]:


def metrics_master(model, X_test, y_test, type):
    metrics_weighted(model, X_test, y_test, type)
    metrics_macro(model, X_test, y_test, type)

def metrics_macro(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    data = {
    "Metric": ["Accuracy", "Macro Precision", "Macro Recall", "Macro F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/XGB.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Macro Precision: {precision * 100:.2f}%")
    print(f"Macro Recall: {recall * 100:.2f}%")
    print(f"Macro F1 Score: {f1 * 100:.2f}%")
    
def metrics_weighted(model, X_test, y_test, type):
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=-1)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    data = {
    "Metric": ["Accuracy", "Weighted Precision", "Weighted Recall", "Weighted F1 Score"],
    type: [accuracy, precision, recall, f1]
    }
    
    # Creating DataFrame
    df = pd.DataFrame(data)
    
    # Saving to CSV
    csv_file_path = "/home/jovyan/A2PM/XGB.csv"
    df.to_csv(csv_file_path, mode='a', index=False)

    print(type)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Weighted Precision: {precision * 100:.2f}%")
    print(f"Weighted Recall: {recall * 100:.2f}%")
    print(f"Weighted F1 Score: {f1 * 100:.2f}%")

## Scores

In [15]:
xgb_model = grid_search_xgb.best_estimator_
metrics_master(xgb_model, X_test, y_test, "original score")

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


original score
Accuracy: 92.71%
Weighted Precision: 85.95%
Weighted Recall: 92.71%
Weighted F1 Score: 89.20%
original score
Accuracy: 92.71%
Macro Precision: 18.54%
Macro Recall: 20.00%
Macro F1 Score: 19.24%


# A2PM Generation

Generate A2PM adversarial samples for further experiments

Since we know the data is pre-processed, the categorical variables will be binary columns, and conversely the numerical variables will be the others. Let's find these columns.

In [8]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

In [9]:
X_train.shape

(716256, 42)

The following configuration is the boilerplate found on the A2PM github (https://github.com/vitorinojoao/a2pm). The only modifications are that the interval pattern is applied to the numerical columns, and the combination pattern is applied to the categorical (binary) variables. I removed integer features since the numerical columns are float values, and removed locked_features as I wasn't sure which features needed to be kept static (data was given as processed .npy files)

In [18]:
xgb_base = grid_search_xgb.best_estimator_

classifier = SklearnWrapper(xgb_base)


# rule of thumb: Interval for numerical, combination for categorical
pattern = (
    {
        "type": "interval",
        "features": numerical_columns,
        #"integer_features": list(range(10, 20)),
        "ratio": 0.1,
        "max_ratio": 0.3,
        "missing_value": 0.0,
        "probability": 1,
    },
    # {
    #     "type": "combination",
    #     "features": binary_columns,
    #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
    #     "probability": 0.4,
    # },
)

method = A2PMethod(pattern)

We will generate attacks based off the entire dataset. We will resplit the train-test sets afterward in our experiments

In [19]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [20]:
start_time = time.time()

X_adversarial = method.fit_generate(classifier, X, y)

training_time = time.time() - start_time
print(f"Attack Time: {training_time}")

Attack Time: 266.8015856742859


# Scores after A2PM Attack

Here we are evaluating how our trained (unprotected) RF model performs on the full A2PM dataset. Each sample is now perturbed, but should still correspond to the original target classes y

In [22]:
metrics_master(xgb_model, X_adversarial, y, "full A2PM Attack")

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


full A2PM Attack
Accuracy: 92.71%
Weighted Precision: 85.95%
Weighted Recall: 92.71%
Weighted F1 Score: 89.20%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


full A2PM Attack
Accuracy: 92.71%
Macro Precision: 18.54%
Macro Recall: 20.00%
Macro F1 Score: 19.24%


# Adversarial Training Preprocess

As per many adversarial defense papers, adversarial training has been cited as one of the best ways to improve resilience. We will retrain our model with the original data, in addition to an added 10% of the data being adversarial samples with an "adversarial" class (class 19).

In [10]:
X_train.shape[0]+X_test.shape[0]

955009

In [11]:

adv_samples_num = int(955009*0.10/0.9) 
adv_samples_num # 10%

106112

In [12]:
X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
X_adversarial_sampled = X_adversarial[X_adversarial_indices]

y_adv = np.full(adv_samples_num, 5)
y_adv

NameError: name 'X_adversarial' is not defined

In [26]:
np.unique(y_train)

array([0, 1, 2, 3, 4])

In [27]:
X_new = X.copy()
y_new = y.copy()

X_combined = np.vstack((X_new, X_adversarial_sampled))
y_combined = np.concatenate((y_new, y_adv))

# Adversarial Training

Our whole data now contains 10% A2PM samples. Assuming a uniformly random 80/20 split, our training data will be expected to contain 8% A2PM samples, while the testing data is expected to contain 2% A2PM samples. 

In [28]:
X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

In [30]:
model_xgb_protected = XGBClassifier()

param_grid = {
    # 'n_estimators': [10, 50, 100],
    # 'max_depth': [None, 50, 100],
    # 'n_estimators': [10],
    # 'max_depth': [None],
}

grid_search_xgb_protected = GridSearchCV(model_xgb_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
grid_search_xgb_protected.fit(X_train_adv, y_train_adv)

print("Best parameters found: ", grid_search_xgb.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   4.2s
[CV 2/3] END ..................................., score=1.000 total time=   4.1s
[CV 3/3] END ..................................., score=1.000 total time=   4.1s
Best parameters found:  {}


In [31]:
xgb_model_protected = grid_search_xgb_protected.best_estimator_

# Scores (Binary A2PM Detection after Adversarial Training)

We want to see how well our protected model can detect A2PM samples. We run it against the unseen A2PM set

In [32]:
X_adversarial_unseen = X_adversarial[np.logical_not(np.isin(np.arange(X_adversarial.shape[0]), X_adversarial_indices))]
len(X_adversarial_unseen)

848897

In [33]:
protected_adversarial_preds = xgb_model_protected.predict(X_adversarial_unseen)

All we need to know is if the model correctly predicts every sample as class 19 (A2PM), and as such we will treat this as a binary classification problem.

In [34]:
y_adv_full = np.full(len(X_adversarial_unseen), 5)

positive_class = 5
y_test_binary = (y_adv_full == positive_class).astype(int)
adversarial_preds_binary = (protected_adversarial_preds == positive_class).astype(int)

In [35]:
adversarial_accuracy = accuracy_score(y_test_binary, adversarial_preds_binary)
adversarial_precision = precision_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_recall = recall_score(y_test_binary, adversarial_preds_binary, average='binary')
adversarial_f1 = f1_score(y_test_binary, adversarial_preds_binary, average='binary')

print(f"Adversarial Accuracy: {adversarial_accuracy * 100:.2f}%")
print(f"Adversarial Precision: {adversarial_precision * 100:.2f}%")
print(f"Adversarial Recall: {adversarial_recall * 100:.2f}%")
print(f"Adversarial F1 Score: {adversarial_f1 * 100:.2f}%")

Adversarial Accuracy: 100.00%
Adversarial Precision: 100.00%
Adversarial Recall: 100.00%
Adversarial F1 Score: 100.00%


# Scores (10% A2PM Samples, no Adv Training)

Here we benchmark how well our unprotected model performs on a test set with 10% A2PM samples and 90% real samples

In [36]:
metrics_master(xgb_model, X_test_adv, y_test_adv, "no Adv Training")

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


no Adv Training
Accuracy: 88.28%
Weighted Precision: 79.68%
Weighted Recall: 88.28%
Weighted F1 Score: 83.52%
no Adv Training
Accuracy: 88.28%
Macro Precision: 81.28%
Macro Recall: 65.63%
Macro F1 Score: 72.03%


# Scores (10% A2PM Samples w/ Adv Training)

Here we benchmark how well our protected model performs on a test set with 10% A2PM samples and 90% real samples

In [38]:
metrics_master(xgb_model_protected, X_test_adv, y_test_adv, "w/ Adv Training")

w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
w/ Adv Training
Accuracy: 100.00%
Macro Precision: 100.00%
Macro Recall: 100.00%
Macro F1 Score: 100.00%


In [39]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

binary_columns = find_binary_columns(X_train)

numerical_columns = []
for i in range(0,42):
    if i not in binary_columns:
        numerical_columns.append(i)

xgb_base = grid_search_xgb.best_estimator_

classifier = SklearnWrapper(xgb_base)


ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    pattern = (
        {
            "type": "interval",
            "features": numerical_columns,
            #"integer_features": list(range(10, 20)),
            "ratio": ratio,
            "max_ratio": 0.3,
            "missing_value": 0.0,
            "probability": 1,
        },
        # {
        #     "type": "combination",
        #     "features": binary_columns,
        #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
        #     "probability": 0.4,
        # },
    )
    
    method = A2PMethod(pattern)
    
    X = np.concatenate((X_train, X_test), axis=0)
    y = np.concatenate((y_train, y_test), axis=0)
    
    start_time = time.time()
    
    X_adversarial = method.fit_generate(classifier, X, y)
    
    training_time = time.time() - start_time
    print(f"Attack Time: {training_time}")

    text = f"{ratio}:full A2PM Attack"
    metrics_master(xgb_model, X_adversarial, y, text)
    
    
    adv_samples_num = int(955009*0.10/0.9) 
    
    X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
    X_adversarial_sampled = X_adversarial[X_adversarial_indices]
    
    y_adv = np.full(adv_samples_num, 5)
    
    X_new = X.copy()
    y_new = y.copy()
    
    X_combined = np.vstack((X_new, X_adversarial_sampled))
    y_combined = np.concatenate((y_new, y_adv))
    
    X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)
    
    model_xgb_protected = XGBClassifier()
    
    param_grid = {
        # 'n_estimators': [10, 50, 100],
        # 'max_depth': [None, 50, 100],
        # 'n_estimators': [10],
        # 'max_depth': [None],
    }
    
    grid_search_xgb_protected = GridSearchCV(model_xgb_protected, param_grid, cv=3, scoring='accuracy', verbose=3)
    grid_search_xgb_protected.fit(X_train_adv, y_train_adv)
    
    print("Best parameters found: ", grid_search_xgb.best_params_)
    
    xgb_model_protected = grid_search_xgb_protected.best_estimator_

    text = f"{ratio}:no Adv Training"
    metrics_master(xgb_model, X_test_adv, y_test_adv, text)
    text = f"{ratio}:w/ Adv Training"
    metrics_master(xgb_model_protected, X_test_adv, y_test_adv, text)

Attack Time: 265.255330324173


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.01:full A2PM Attack
Accuracy: 92.71%
Weighted Precision: 85.95%
Weighted Recall: 92.71%
Weighted F1 Score: 89.20%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.01:full A2PM Attack
Accuracy: 92.71%
Macro Precision: 18.54%
Macro Recall: 20.00%
Macro F1 Score: 19.24%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   4.3s
[CV 2/3] END ..................................., score=1.000 total time=   4.2s
[CV 3/3] END ..................................., score=1.000 total time=   4.2s
Best parameters found:  {}
0.01:no Adv Training
Accuracy: 88.28%
Weighted Precision: 79.68%
Weighted Recall: 88.28%
Weighted F1 Score: 83.52%


...

0.3:full A2PM Attack
Accuracy: 92.71%
Weighted Precision: 85.95%
Weighted Recall: 92.71%
Weighted F1 Score: 89.20%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3:full A2PM Attack
Accuracy: 92.71%
Macro Precision: 18.54%
Macro Recall: 20.00%
Macro F1 Score: 19.24%
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END ..................................., score=1.000 total time=   4.1s
[CV 2/3] END ..................................., score=1.000 total time=   4.1s
[CV 3/3] END ..................................., score=1.000 total time=   4.2s
Best parameters found:  {}
0.3:no Adv Training
Accuracy: 88.28%
Weighted Precision: 79.68%
Weighted Recall: 88.28%
Weighted F1 Score: 83.52%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3:no Adv Training
Accuracy: 88.28%
Macro Precision: 81.28%
Macro Recall: 65.63%
Macro F1 Score: 72.03%
0.3:w/ Adv Training
Accuracy: 100.00%
Weighted Precision: 100.00%
Weighted Recall: 100.00%
Weighted F1 Score: 100.00%
0.3:w/ Adv Training
Accuracy: 100.00%
Macro Precision: 99.99%
Macro Recall: 100.00%
Macro F1 Score: 99.99%


In [14]:
# generate a2pm sample

ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    pattern = (
        {
            "type": "interval",
            "features": numerical_columns,
            #"integer_features": list(range(10, 20)),
            "ratio": ratio,
            "max_ratio": 0.3,
            "missing_value": 0.0,
            "probability": 1,
        },
        # {
        #     "type": "combination",
        #     "features": binary_columns,
        #     #"locked_features": list(range(30, 40)), # Locks some features to ensure validity. Not using this because data is .npy and unreadable
        #     "probability": 0.4,
        # },
    )
    
    method = A2PMethod(pattern)
    
    X = np.concatenate((X_train, X_test), axis=0)
    y = np.concatenate((y_train, y_test), axis=0)
    
    start_time = time.time()
    
    X_adversarial = method.fit_transform(X, y)
    
    training_time = time.time() - start_time
    print(f"Attack Time: {training_time}")

    # text = f"{ratio}:full A2PM Attack"
    # metrics_master(xgb_model, X_adversarial, y, text)
    text = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_X"
    np.save(text, X_adversarial)
    text = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_y"
    np.save(text, y)
    
    
    adv_samples_num = int(955009*0.10/0.9) 
    
    X_adversarial_indices = np.random.choice(X_adversarial.shape[0], size=adv_samples_num, replace=False)
    X_adversarial_sampled = X_adversarial[X_adversarial_indices]
    
    y_adv = np.full(adv_samples_num, 5)
    
    X_new = X.copy()
    y_new = y.copy()
    
    X_combined = np.vstack((X_new, X_adversarial_sampled))
    y_combined = np.concatenate((y_new, y_adv))

    text = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_10%attack_X"
    np.save(text, X_combined)
    text = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_10%attack_y"
    np.save(text, y_combined)
    
    # X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)
    

Attack Time: 80.18626546859741
Attack Time: 80.38876819610596
Attack Time: 80.09422469139099
Attack Time: 79.98427414894104
Attack Time: 64.89600849151611


In [6]:
X_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
X_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
X_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

dnn_model = Sequential()
dnn_model.add(Input(shape=input_shape))
dnn_model.add(Dense(units=30, activation='relu'))
dnn_model.add(Dense(units=20, activation='relu'))
dnn_model.add(Dense(units=num_classes, activation='softmax'))

opt = SGD(learning_rate=0.01)

dnn_model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
hist = dnn_model.fit(X_train, y_train, epochs=10, batch_size=64,
                     validation_data=(X_val, y_val),
                     callbacks=[early_stopping])

ratio_values = [0.01, 0.05, 0.1, 0.2, 0.3]
# rule of thumb: Interval for numerical, combination for categorical
for ratio in ratio_values:
    xtext = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_X.npy"
    X_adv = np.load(xtext)
    ytext = f"/home/jovyan/A2PM/A2PM_adv_sample/A2PM_{ratio}_fullattack_y.npy"
    y_adv = np.load(ytext)
    
    X_train_adv, X_test_adv, y_train_adv, y_test_adv = train_test_split(X_adv, y_adv, test_size=0.2, random_state=42)
    

    text=f"{ratio}_origin"
    metrics_master(dnn_model, X_test, y_test, text)

    text=f"{ratio}_test attack dataset"
    metrics_master(dnn_model, X_test_adv, y_test_adv, text)

    
    input_shape = X_train_adv.shape[1:]
    num_classes = len(np.unique(y_train_adv))
    
    dnn_model_def = Sequential()
    dnn_model_def.add(Input(shape=input_shape))
    dnn_model_def.add(Dense(units=30, activation='relu'))
    dnn_model_def.add(Dense(units=20, activation='relu'))
    dnn_model_def.add(Dense(units=num_classes, activation='softmax'))
    
    opt = SGD(learning_rate=0.01)
    
    dnn_model_def.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
    hist = dnn_model_def.fit(X_train_adv, y_train_adv, epochs=10, batch_size=64,
                         validation_data=(X_train_adv, y_train_adv),
                         callbacks=[early_stopping])


    text=f"{ratio}_defensed model"
    metrics_master(dnn_model_def, X_test_adv, y_test_adv, text)

Epoch 1/10


2024-07-10 19:08:10.628845: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_SYSTEM_NOT_READY: system not yet initialized
2024-07-10 19:08:10.628883: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: jupyter-jic128-40ucsd-2eedu
2024-07-10 19:08:10.628889: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: jupyter-jic128-40ucsd-2eedu
2024-07-10 19:08:10.629004: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 550.90.7
2024-07-10 19:08:10.629021: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 550.90.7
2024-07-10 19:08:10.629026: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 550.90.7


11192/11192 ━━━━━━━━━━━━━━━━━━━━ 7s 575us/step - accuracy: 0.9786 - loss: 0.1000 - val_accuracy: 0.9934 - val_loss: 0.0154
Epoch 2/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 559us/step - accuracy: 0.9955 - loss: 0.0122 - val_accuracy: 0.9984 - val_loss: 0.0061
Epoch 3/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 569us/step - accuracy: 0.9989 - loss: 0.0051 - val_accuracy: 0.9994 - val_loss: 0.0036
Epoch 4/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 565us/step - accuracy: 0.9994 - loss: 0.0033 - val_accuracy: 0.9995 - val_loss: 0.0027
Epoch 5/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 563us/step - accuracy: 0.9995 - loss: 0.0027 - val_accuracy: 0.9996 - val_loss: 0.0022
Epoch 6/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 563us/step - accuracy: 0.9996 - loss: 0.0021 - val_accuracy: 0.9996 - val_loss: 0.0019
Epoch 7/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 553us/step - accuracy: 0.9996 - loss: 0.0018 - val_accuracy: 0.9996 - val_loss: 0.0017
Epoch 8/10
11192/11192 ━━━━━━━━━━━━━━━━━━━━ 6s 574us/step - accuracy: 0.9

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5969/5969 ━━━━━━━━━━━━━━━━━━━━ 2s 399us/step


...

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3_test attack dataset
Accuracy: 99.88%
Macro Precision: 68.54%
Macro Recall: 62.16%
Macro F1 Score: 62.74%
Epoch 1/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 828us/step - accuracy: 0.9748 - loss: 0.1082 - val_accuracy: 0.9931 - val_loss: 0.0142
Epoch 2/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 814us/step - accuracy: 0.9952 - loss: 0.0122 - val_accuracy: 0.9981 - val_loss: 0.0082
Epoch 3/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 811us/step - accuracy: 0.9984 - loss: 0.0075 - val_accuracy: 0.9989 - val_loss: 0.0056
Epoch 4/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 810us/step - accuracy: 0.9990 - loss: 0.0053 - val_accuracy: 0.9991 - val_loss: 0.0044
Epoch 5/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 803us/step - accuracy: 0.9991 - loss: 0.0043 - val_accuracy: 0.9991 - val_loss: 0.0037
Epoch 6/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 794us/step - accuracy: 0.9991 - loss: 0.0038 - val_accuracy: 0.9991 - val_loss: 0.0033
Epoch 7/10
11938/11938 ━━━━━━━━━━━━━━━━━━━━ 10s 799us/step - accuracy: 0.9992 - l

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


5969/5969 ━━━━━━━━━━━━━━━━━━━━ 2s 399us/step
0.3_defensed model
Accuracy: 99.94%
Macro Precision: 59.36%
Macro Recall: 59.59%
Macro F1 Score: 59.48%


/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
